# Sistema de Assistência Virtual

Desenvolvindo por: **Bruno Ferro**

Segue código pronto para Colab que implementa um sistema de assistência virtual mínimo conforme o desafio do Módulo 6: módulos TTS, STT (upload de áudio e gravação via navegador), intent matcher por palavras-chave e handlers para abrir Wikipédia, YouTube e localizar farmácia mais próxima (gera links/googles). Código comentado e direto ao ponto.

Observação operacional: em Colab não é possível abrir janelas do navegador automaticamente; o notebook gera links clicáveis e reproduz áudio no próprio ambiente. A cidade estimada usada para procurar a farmácia é São Paulo, SP, Brazil (informação do ambiente).
Código fornecido abaixo — cole e execute as células em ordem no Colab.

# Colab-ready: Assistente Virtual (TTS, STT, intent handlers: Wikipedia, YouTube, "farmácia mais próxima")

In [9]:
# Requisitos: speechrecognition, gTTS, wikipedia, pydub, ipywidgets
!pip install -q SpeechRecognition gTTS wikipedia pydub ipywidgets

# 1) Imports e configurações

In [10]:
import os, io, re, urllib.parse
from IPython.display import Audio, display, HTML
from gtts import gTTS
import speech_recognition as sr
import wikipedia
from pydub import AudioSegment

# --- Ajustes ---

In [31]:
# Cidade estimada (obtida via user_info tool): São Paulo, SP, Brazil
DEFAULT_CITY = "São Paulo, SP, Brazil"
# LANG = "pt-BR"  # linguagem para TTS/STT/wikipedia
LANG = "pt"  # linguagem para TTS/STT/wikipedia
wikipedia.set_lang("pt")

Exemplos práticos para LANG

"pt-BR" → Português do Brasil

"en-US" → Inglês (Estados Unidos)

"es-ES" → Espanhol (Espanha)

"fr-FR" → Francês (França)

"de-DE" → Alemão

AVISO: gtts.lang: 'pt-BR' foi descontinuado, voltando para 'pt'. Este fallback será removido em uma versão futura.

# 2) Módulo Text-to-Speech: sintetiza texto -> reproduz no Colab e salva opcionalmente

In [ ]:
# def tts_play(text: str, lang: str = LANG, filename: str = None):
#     """Sintetiza e reproduz áudio do texto (gTTS). Retorna bytes do mp3."""
#     tts = gTTS(text=text, lang=lang.replace("-","_") if "-" in lang else lang)
#     buf = io.BytesIO()
#     tts.write_to_fp(buf)
#     buf.seek(0)
#     if filename:
#         with open(filename, "wb") as f:
#             f.write(buf.getbuffer())
#     display(Audio(buf.read(), autoplay=True))
#     buf.seek(0)
#     return buf.read()

In [32]:
def tts_play(text: str, lang: str = LANG, filename: str = None):
    """Sintetiza e reproduz áudio do texto (gTTS). Retorna bytes do mp3."""
    tts = gTTS(text=text, lang=lang) # Removed .replace("-","_")
    buf = io.BytesIO()
    tts.write_to_fp(buf)
    buf.seek(0)
    if filename:
        with open(filename, "wb") as f:
            f.write(buf.getbuffer())
    display(Audio(buf.read(), autoplay=True))
    buf.seek(0)
    return buf.read()

# 3) Módulo Speech-to-Text (arquivo WAV/MP3 uploadado)

In [33]:
def stt_from_file(path: str, lang: str = LANG):
    """Recebe caminho para arquivo de áudio (wav/mp3) e retorna texto reconhecido."""
    r = sr.Recognizer()
    # Converte para WAV 16k mono se necessário (pydub)
    audio_path = path
    if not path.lower().endswith(".wav"):
        # converte para wav
        audio = AudioSegment.from_file(path)
        tmp_wav = "/tmp/temp_stt.wav"
        audio.set_frame_rate(16000).set_channels(1).export(tmp_wav, format="wav")
        audio_path = tmp_wav
    with sr.AudioFile(audio_path) as source:
        audio_data = r.record(source)
    try:
        # usa recognizer offline pocketsphinx se instalado, caso contrário Google Web Speech API (online)
        text = r.recognize_google(audio_data, language=lang)
        return text
    except sr.UnknownValueError:
        return ""
    except sr.RequestError as e:
        # erro de rede ou quota -> propagar mensagem
        raise RuntimeError(f"API error: {e}")

# 4) Módulo para gravar via navegador (gera /content/recorded.wav)

In [34]:
record_js = """
<button id="record">Gravar 5s</button>
<script>
const btn = document.getElementById('record');
btn.onclick = async () => {
  btn.disabled = true;
  const stream = await navigator.mediaDevices.getUserMedia({ audio: true });
  const mediaRecorder = new MediaRecorder(stream);
  const chunks = [];
  mediaRecorder.ondataavailable = e => chunks.push(e.data);
  mediaRecorder.onstop = async () => {
    const blob = new Blob(chunks, { type: 'audio/webm' });
    const arrayBuffer = await blob.arrayBuffer();
    const bytes = new Uint8Array(arrayBuffer);
    // send to python
    const base64 = btoa(String.fromCharCode(...bytes));
    const py = `
from base64 import b64decode
b = b64decode("${base64}")
open("/content/recorded.webm","wb").write(b)
print("saved /content/recorded.webm")
`;
    google.colab.kernel.invokeFunction('notebook.RunPython', [py], {});
    btn.disabled = false;
  };
  mediaRecorder.start();
  setTimeout(() => mediaRecorder.stop(), 5000); // 5s gravação
}
</script>
"""
# helper to connect JS->python (Colab)
from google.colab import output
def _run_py_code(code):
    exec(code, globals())
output.register_callback('notebook.RunPython', _run_py_code)

def show_recorder_widget():
    """Mostra botão para gravar ~5s no navegador (salva /content/recorded.webm)."""
    display(HTML(record_js))
    print("Clique em 'Gravar 5s'. Ao fim, execute stt_from_recorded()")

def stt_from_recorded(lang: str = LANG):
    """Converte /content/recorded.webm -> text via stt_from_file"""
    webm = "/content/recorded.webm"
    if not os.path.exists(webm):
        raise FileNotFoundError("Arquivo /content/recorded.webm não encontrado. Use show_recorder_widget() primeiro.")
    # converte webm -> wav
    audio = AudioSegment.from_file(webm)
    tmp_wav = "/content/recorded.wav"
    audio.set_frame_rate(16000).set_channels(1).export(tmp_wav, format="wav")
    return stt_from_file(tmp_wav, lang=lang)

# 5) Intent matching (simples, baseado em palavras-chave)

In [35]:
def parse_intent(text: str):
    """Retorna (intent, params). Intents: wiki_search, youtube_search, find_pharmacy, tts_playback, unknown"""
    txt = text.lower()
    # wikipedia intent
    m = re.search(r'\b(wikipedia|wik[píi]a|pesquise no wikipedia|pesquisar no wikipedia)\b', txt)
    if m:
        # tenta extrair tópico após 'sobre' ou 'pesquise ...'
        topic = None
        # busca "sobre X"
        mm = re.search(r'(?:sobre|sobre o|sobre a)\s+(.+)', text, re.IGNORECASE)
        if mm:
            topic = mm.group(1).strip()
        else:
            # fallback: pega últimas 6 palavras
            topic = " ".join(text.split()[-6:])
        return ("wiki_search", {"topic": topic})
    # youtube search
    if any(k in txt for k in ["youtube", "youtub", "vídeo", "toque no youtube", "abrir youtube", "pesquisar no youtube"]):
        # extrair termo
        mm = re.search(r'(?:youtube|vídeo|pesquisar no youtube|abrir youtube)\s*(?:sobre|de|:)?\s*(.*)', text, re.IGNORECASE)
        term = mm.group(1).strip() if mm and mm.group(1).strip() else text
        return ("youtube_search", {"term": term})
    # find pharmacy
    if any(k in txt for k in ["farmácia", "farmacia", "drogaria", "remédio próximo", "farmácia mais próxima", "drogaria mais próxima"]):
        return ("find_pharmacy", {"city": DEFAULT_CITY})
    # tts playback (user asked to speak)
    if any(k in txt for k in ["diga", "fale", "falar", "lê", "ler"]):
        # tenta extrair texto entre aspas ou após comando
        mm = re.search(r'["“](.+?)["”]', text)
        snippet = mm.group(1) if mm else re.sub(r'.*(diga|fale|ler|lê)\s*', '', text, flags=re.IGNORECASE)
        return ("tts_playback", {"text": snippet.strip()})
    return ("unknown", {})

# 6) Handlers: executar ação para cada intent

In [36]:
def handle_wiki(topic: str, sentences: int = 3):
    """Busca sumário no Wikipedia e retorna texto + link"""
    if not topic:
        return {"text": "Tópico para Wikipédia não especificado.", "link": None}
    try:
        summary = wikipedia.summary(topic, sentences=sentences)
        page = wikipedia.page(topic, auto_suggest=True, redirect=True)
        link = page.url
        return {"text": summary, "link": link}
    except Exception as e:
        return {"text": f"Erro ao buscar no Wikipédia: {e}", "link": None}

def handle_youtube(term: str):
    """Gera link de busca no YouTube e frase de confirmação"""
    if not term:
        term = ""
    q = urllib.parse.quote_plus(term)
    url = f"https://www.youtube.com/results?search_query={q}"
    return {"text": f"Abrindo pesquisa no YouTube para: {term}", "link": url}

def handle_find_pharmacy(city: str = DEFAULT_CITY):
    """Gera Google Maps search para farmácia perto da cidade"""
    q = urllib.parse.quote_plus(f"farmácia perto de {city}")
    url = f"https://www.google.com/maps/search/{q}"
    return {"text": f"Localizando farmácias próximas em {city}", "link": url}

# 7) Pipeline principal: recebe texto (STT) -> parse_intent -> handler -> TTS + retorno de links

In [37]:
def assistant_pipeline_from_text(text: str, speak_result: bool = True):
    """Executa pipeline completo a partir de texto reconhecido."""
    intent, params = parse_intent(text)
    if intent == "wiki_search":
        res = handle_wiki(params.get("topic",""))
        if speak_result:
            tts_play(res["text"])
        out_html = f"<b>Wikipédia (tópico: {params.get('topic','-')}):</b><br>{res['text'][:800]}<br>"
        if res.get("link"):
            out_html += f'<a href="{res["link"]}" target="_blank">Abrir página no Wikipédia</a>'
        display(HTML(out_html))
        return res
    elif intent == "youtube_search":
        res = handle_youtube(params.get("term",""))
        if speak_result:
            tts_play(res["text"])
        display(HTML(f'<a href="{res["link"]}" target="_blank">{res["text"]} — abrir YouTube</a>'))
        return res
    elif intent == "find_pharmacy":
        res = handle_find_pharmacy(params.get("city", DEFAULT_CITY))
        if speak_result:
            tts_play(res["text"])
        display(HTML(f'<a href="{res["link"]}" target="_blank">{res["text"]} — abrir Maps</a>'))
        return res
    elif intent == "tts_playback":
        txt = params.get("text","")
        if not txt: txt = "Comando de fala sem conteúdo."
        tts_play(txt)
        return {"text": txt}
    else:
        tts_play("Desculpe, não entendi o comando.")
        display(HTML(f"<b>Intent desconhecida.</b> Texto recebido: {text}"))
        return {"text": "", "unknown": True}

In [38]:
# 8) Exemplos de uso (instruções)
print("Células prontas. Exemplos de uso:\n"
      "1) Gravar via navegador: show_recorder_widget() -> depois stt = stt_from_recorded() -> assistant_pipeline_from_text(stt)\n"
      "2) Fazer upload de arquivo de áudio (wav/mp3) e usar stt_from_file('/path/file.wav') -> assistant_pipeline_from_text(stt)\n"
      "3) Sintetizar texto: tts_play('Olá pesquisador')\n")

Células prontas. Exemplos de uso:
1) Gravar via navegador: show_recorder_widget() -> depois stt = stt_from_recorded() -> assistant_pipeline_from_text(stt)
2) Fazer upload de arquivo de áudio (wav/mp3) e usar stt_from_file('/path/file.wav') -> assistant_pipeline_from_text(stt)
3) Sintetizar texto: tts_play('Olá pesquisador')



# Exemplo runtime (comentado para não executar automaticamente)

In [39]:
show_recorder_widget()

Clique em 'Gravar 5s'. Ao fim, execute stt_from_recorded()
saved /content/recorded.webm
saved /content/recorded.webm
saved /content/recorded.webm
saved /content/recorded.webm


# after recording:

In [46]:
text = stt_from_recorded();
print('STT:', text);

STT: Abrir YouTube


In [47]:
assistant_pipeline_from_text(text)

{'text': 'Abrindo pesquisa no YouTube para: Abrir YouTube',
 'link': 'https://www.youtube.com/results?search_query=Abrir+YouTube'}

# Notas técnicas (curtas)



speech_recognition com recognize_google exige conexão; para solução offline recomenda-se pocketsphinx (instalação adicional) ou modelos locais (VOSK).

gTTS usa Google Translate TTS (requisição HTTP). Para uso offline/produção, considerar pyttsx3 ou modelos TTS locais (Tacotron+WaveGlow, etc.).

O intent matcher é keyword-based (simples). Para produção substitua por um classificador de intenção (transformer/Rasa/spaCy) com NLU e extração de entidades para robustez.

# Referências (3)



Python SpeechRecognition library — https://pypi.org/project/SpeechRecognition/

gTTS (Google Text-to-Speech) — https://pypi.org/project/gTTS/

Wikipedia API for Python — https://pypi.org/project/wikipedia/